In [78]:
import tensorflow as tf

In [79]:
# Building a simple dataset
daily_sales_number = [21,22,-108,31,-1,32,34,31]

In [80]:
# Building the tf dataset
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [81]:
# View the content of TF_Dataset as numpy
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [82]:
# Viewing first few elements as numpy format

for sales in tf_dataset.take(3):
    print(sales.numpy())


21
22
-108


In [83]:
# Filtering the sales number as they can't be negative

tf_dataset = tf_dataset.filter(lambda x:x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

# It will remove any negative value

21
22
31
32
34
31


In [84]:
# For example these number are in usd so we can convert them to inr
# using map function

tf_dataset = tf_dataset.map(lambda x:x*83)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1743
1826
2573
2656
2822
2573


In [85]:
# We can also suffle the elements

tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1826
1743
2822
2573
2656
2573


In [86]:
# We can also do batching if we runing on multi gpu envirement
for sales_batch in tf_dataset.batch(4):
    print(sales_batch)

tf.Tensor([2573 1826 1743 2656], shape=(4,), dtype=int32)
tf.Tensor([2573 2822], shape=(2,), dtype=int32)


In [87]:
# Doing all the operation in one single line

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_number)

tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda y:y*83).shuffle(3).batch(2)

for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[2573 1743]
[1826 2573]
[2656 2822]


In [88]:
# Loading cat & dog images for diff operarion
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle = False)

In [89]:
# reading the files
for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/055ba2dc-3a2b-421f-9ab4-a4ddd8d89485.jpeg'
b'images/cat/22a7b0e6-47d2-4196-932d-060c7b648de5.jpeg'
b'images/cat/50bfeef9-7b07-4491-9c0b-4de5972f2298.jpeg'


In [90]:
# Shuffling the files
images_ds = images_ds.shuffle(200)

for file in images_ds.take(3):
    print(file.numpy())

b'images/cat/download (6).jpeg'
b'images/dog/download (4) 4.20.35\xe2\x80\xafAM.jpeg'
b'images/cat/055ba2dc-3a2b-421f-9ab4-a4ddd8d89485.jpeg'


In [91]:
# Creating class of cat & dog
class_names = ["cat","dog"]

In [92]:
image_count = len(images_ds)
image_count

41

**Now we'll divide it into train & test sample.**

**Earlier we were using the sklearn train test split, but in this we use take function**

In [93]:
# Def train size as 80 percent
train_size = int(image_count*0.8)

# Using take function
train_ds = images_ds.take(train_size)

# Using skip its oposite of take
test_ds = images_ds.skip(train_size)

In [94]:
len(train_ds)

32

In [95]:
len(test_ds)

9

In [96]:
# How we can Retrieve the image Label from image path
s = 'images/dog/download (7) 4.20.35\xe2\x80\xafAM.jpeg'

# Python split func to give string at -2 positon
s.split("/")[-2]

'dog'

In [97]:
# Define a function to extract the label from the file path
def get_label(file_path):
    # Import the os module to work with file paths
    import os
    # Split the file path into its components and return the second last component
    # This component corresponds to the class label (e.g., 'cat' or 'dog')
    return tf.strings.split(file_path, os.path.sep)[-2]

In [98]:
# Define a function to process an image file path
def process_image(file_path):
    # Get the label from the file path using the get_label function
    label = get_label(file_path)

    # Read the image file from the file path
    img = tf.io.read_file(file_path)
    # Decode the image as a JPEG file
    img = tf.image.decode_jpeg(img)
    # Resize the image to 128x128 pixels
    img = tf.image.resize(img, [128, 128])

    # Return the processed image and its label
    return img, label

In [99]:
# Apply the process_image function to each file path in the training dataset
# Take the first 3 elements from the resulting dataset and iterate over them
train_ds=train_ds.map(process_image)
for img, label in train_ds.take(3):
    # Print the processed image (as a tensor)
    print("Image: ", img)
    # Print the label (as a tensor)
    print("Label: ", label)

Image:  tf.Tensor(
[[[ 0.         1.         0.       ]
  [ 0.         1.         0.       ]
  [ 0.         1.         0.       ]
  ...
  [ 6.         1.         0.       ]
  [ 6.         1.         0.       ]
  [ 6.         1.         0.       ]]

 [[ 0.         1.         0.       ]
  [ 0.         1.         0.       ]
  [ 0.         1.         0.       ]
  ...
  [ 6.         1.         0.       ]
  [ 6.         1.         0.       ]
  [ 6.         1.         0.       ]]

 [[ 0.         1.         0.       ]
  [ 0.         1.         0.       ]
  [ 0.         1.         0.       ]
  ...
  [ 6.         1.         0.       ]
  [ 6.         1.         0.       ]
  [ 6.         1.         0.       ]]

 ...

 [[42.89868   24.289307   8.461182 ]
  [51.966553  33.357178  18.140625 ]
  [60.545166  42.08838   29.174805 ]
  ...
  [43.796875  27.140625  20.055176 ]
  [61.36377   43.029053  36.92627  ]
  [66.51343   48.984375  42.075928 ]]

 [[34.702637  15.702637   1.5258789]
  [51.695557  32.6

In [100]:
# Scaling the numpy array
def scale(Image, label):
    return Image/255, label

In [110]:
# Apply the scale function to each element in the training dataset
train_ds = train_ds.map(scale)

# Iterate over the first 5 elements of the scaled training dataset
for image, label in train_ds.take(5):
    # Print the value of the top-left pixel (0, 0) of the first image in the batch
    print("Image: ", image.numpy()[0][0])
    # Print the label as a numpy array
    print("Label: ", label.numpy())


Image:  [0.000000e+00 8.601977e-25 0.000000e+00]
Label:  b'dog'
Image:  [2.0644741e-23 1.8064151e-23 1.3763163e-23]
Label:  b'dog'
Image:  [1.4054823e-22 1.3710743e-22 1.2764525e-22]
Label:  b'cat'
Image:  [1.41273995e-22 1.04285506e-22 9.65437385e-23]
Label:  b'dog'
Image:  [2.0533321e-22 1.8459712e-22 1.5539608e-22]
Label:  b'cat'
